# Classification of full NPCs according to their profiles

We want to classify the NPCs within 4 classes: 
1. **"Empty"** : no density is visible in the central channel
2. **"Low basket"** : a basket is visible and its appex is lower than the ring
3. **"High basket"** : a basket is visible and its appex is protruding above the ring
4. **"Trash"** : all structures that do not fit anywhere...



**Tasks to perform:**
- Load the averaged height profiles of all NPCs
- for each NPC
    - store the NPC ID
    - trace the mean profile from -73.5 to +73.5nm
    - fit the ring => two peaks
        - if no local max inbetween => #1
        - if local max with height ≤ ring => #2
        - if local max > ring => #3
        - else => #4
    - store the NPC ID in the corresponding class
 - Once this is done, visually check the classes (both NPC images and NPC profiles)
 

In [1]:
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as img
import matplotlib.gridspec as gridspec

In [3]:
# Load the averaged height profiles of all NPCs

NPC_Profile=pd.read_csv('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/NPC_Profile-2.txt')
print(NPC_Profile.index)


RangeIndex(start=0, stop=84, step=1)


**Here I need to find how to extract the length (nb of NPCs) of NPC_Profile**

NB: to extract local maxima and minima from a 1D array, we are going to use the **argrelmax** (or **argrelmin**) from **scipy.signal**. 

Of note, it retrieves **the index** of local max, not the value. In our configuration, the ring's crest is expected for x around ±42nm, which corresponds to **indexes 3 & 11**.

To get the value, we will have to get back to y[index].

-----

Here, from the position and relative values of the local maxima, we will sort the NPCs within the 4 classes. To retrieve the analysis results, we will store **the indices (from 0 to 83)** of the pore into 4 lists (empty, low_basket, high_basket, trash). **To get the corresponding image indices, one has to add 1.**


In [ ]:
from scipy.signal import argrelmax
from scipy.signal import argrelmin


x=np.linspace(-73.5, 73.5, 15) 
#print(x)

# initialize classes
empty = []
low_basket=[]
high_basket=[]
trash=[]

for p in range (0,84):
    
    id = str(p+1)
    y = NPC_Profile.iloc[p,slice(5,20)].values
    image = img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    
    #fig,(ax0,ax1) = plt.subplots(nrows=1, ncols=2, figsize=(6,3))
    #ax0.imshow(image)
    #ax1.plot(x,y)
    
    # find local max
    Max = argrelmax(y)
    m=Max[-1]
    
   
    #print (id,m)
    
    # 2 local maxima detected
    if len(m) == 2:
        if (m[0]==2 or m[0]==3) and (m[-1]==11 or m[-1]==12):            
            #print(p,'NPC '+id,'Empty')
            empty.append(p)            
        else:
            #print(p,'NPC '+id,'Trash')
            trash.append(p)
                   
    
    # 3 local maxima detected
    elif len(m) == 3:        
        if y[m[1]]<y[m[0]] and y[m[1]]<y[m[-1]]:
            #print(p,'NPC '+id,'Low basket')
            low_basket.append(p)
        else:
            #print(p,'NPC '+id,'High basket')
            high_basket.append(p)
             
    
    # else go to trash
    else:
        #print(p,'NPC '+id,'Trash')
        trash.append(p)
    
    plt.show()
print('empty contains ', len(empty), ' NPCs')
print('Low basket contains ',len(low_basket), ' NPCs')
print('High basket contains ',len(high_basket), ' NPCs')
print('Trash contains ', len(trash), ' NPCs')
Nr = max(len(empty),len(low_basket),len(high_basket),len(trash))
print(Nr)
Ntotal = len(empty)+len(low_basket)+len(high_basket)+len(trash)
print('Ntotal = ', Ntotal)

**Visual evaluation of the classification**

I would like to sort NPC images and height profiles in 4 columns corresponding to the 4 classes. Here is a first trial using subplots and gridspec.

A few notes before going on:
1. some "empty" NPCs end up in the trash. Instead of filtering on the position of local maxima, try filtering on the diameter (difference between m[0] and m[-1].
2. I cannot run the figure script on the entire set of figure. => not enough RAM ? 
3. Try to make independent figures for each class, save them and mount them in a final subplot.

In [8]:
# initialize classes
empty = []
low_basket=[]
high_basket=[]
trash=[]

for p in range (0,84):
    
    id = str(p+1)
    y = NPC_Profile.iloc[p,slice(5,20)].values
    image = img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    
      
    # find local max
    Max = argrelmax(y)
    m=Max[-1]
    
    
    # 2 local maxima detected
    if len(m) == 2:
        if (m[-1]-m[0])>7 and (m[-1]-m[0])<11:            
            empty.append(p)            
        else:
            trash.append(p)
                   
    
    # 3 local maxima detected
    elif len(m) == 3:        
        if (m[-1]-m[0])<8 or (m[-1]-m[0])>10:
            trash.append(p)
        elif y[m[1]]<y[m[0]] and y[m[1]]<y[m[-1]]:
            low_basket.append(p)
        else:
            high_basket.append(p)
             
    
    # else go to trash
    else:
        trash.append(p)
    

print('empty contains ', len(empty), ' NPCs')
print('Low basket contains ',len(low_basket), ' NPCs')
print('High basket contains ',len(high_basket), ' NPCs')
print('Trash contains ', len(trash), ' NPCs')
Nr = max(len(empty),len(low_basket),len(high_basket),len(trash))
print(Nr)


empty contains  35  NPCs
Low basket contains  25  NPCs
High basket contains  13  NPCs
Trash contains  11  NPCs
35
Ntotal =  84


In [ ]:
Nc = 8


fig, axs = plt.subplots(nrows=Nr, ncols=Nc, constrained_layout=True, figsize=(16,2*Nr))
fig.suptitle('NPC classes')

#images=[]
N=len(empty)
for n in range(0,N):
    p=empty[n]
    id = str(p+1)
    
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    

    axs[n,0].imshow(image)
    axs[n,1].plot(x,y)
    
    

    
N=len(low_basket)
for n in range (0,N):
    p=low_basket[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
        
    
    axs[n,2].imshow(image)
    axs[n,3].plot(x,y)

    
    
N=len(high_basket)
for n in range (0,N):
    p=high_basket[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    
    
    axs[n,4].imshow(image)
    axs[n,5].plot(x,y)
    
   
    
N=len(trash)
for n in range (0,N):
    p=trash[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
        
    
    axs[n,6].imshow(image)
    axs[n,7].plot(x,y)
       





In [11]:
N=len(empty)

fig0, ax0 = plt.subplots(nrows=N, ncols=2, constrained_layout=True, figsize=(4,2*N))
fig0.suptitle('Empty '+string(100*len(empty)/Ntotal)+' %', fontsize=36)

#images=[]

for n in range(0,N):
    p=empty[n]
    id = str(p+1)
    
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    
    
    ax0[n,0].imshow(image)
    ax0[n,0].axis('off')
    ax0[n,1].plot(x,y)

plt.savefig('empty.tif')    
    

    
N=len(low_basket)

fig1, ax1=plt.subplots(nrows=N, ncols=2, constrained_layout=True, figsize=(4,2*N))
fig1.suptitle('Low basket', fontsize=36)

for n in range (0,N):
    p=low_basket[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
        
    
    ax1[n,0].imshow(image)
    ax1[n,0].axis('off')
    ax1[n,1].plot(x,y)

plt.savefig('low_basket.tif')

    
    
N=len(high_basket)

fig2, ax2=plt.subplots(nrows=N, ncols=2, constrained_layout=True, figsize=(4,2*N))
fig2.suptitle('High basket', fontsize=36)

for n in range (0,N):
    p=high_basket[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
    
    
    ax2[n,0].imshow(image)
    ax2[n,0].axis('off')
    ax2[n,1].plot(x,y)

plt.savefig('high_basket.tif')
 
   
    
N=len(trash)

fig3, ax3=plt.subplots(nrows=N, ncols=2, constrained_layout=True, figsize=(4,2*N))
fig3.suptitle('Trash', fontsize=36)

for n in range (0,N):
    p=trash[n]
    id = str(p+1)
    y=NPC_Profile.iloc[p,slice(5,20)].values
    image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/2um/2um_crop/2um_NPC_'+id+'.tif')
        
    
    ax3[n,0].imshow(image)
    ax3[n,0].axis('off')
    ax3[n,1].plot(x,y)
    
plt.savefig('trash.tif')


       

NameError: name 'string' is not defined

Error in callback <function flush_figures at 0x7fa6c88c4550> (for post_execute):


KeyboardInterrupt: 

In [ ]:
from matplotlib.gridspec import GridSpec

fig = plt.figure(tight_layout=True, figsize=(16,2*Nr+2))
fig.suptitle('NPC classes', fontsize=36)
gs = GridSpec(Nr+1, 4, figure=fig)


#fig,axs=plt.subplots(nrows=1,ncols=4,tight_layout=True,figsize=(4, Nr))


ax0 = fig.add_subplot(gs[1:, 0], anchor='N')
image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/empty.tif')
ax0.imshow(image)
ax0.axis('off')

ax1=fig.add_subplot(gs[1:,1],anchor='N')
image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/low_basket.tif')
ax1.imshow(image)
ax1.axis('off')

ax2=fig.add_subplot(gs[1:,2],anchor='N')
image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/high_basket.tif')
ax2.imshow(image)
ax2.axis('off')

ax3=fig.add_subplot(gs[1:,3],anchor='N')
image=img.imread('/Users/christine/Documents/Data/AFM/Full NPC Databank/trash.tif')
ax3.imshow(image)
ax3.axis('off')

plt.savefig('NPC_classes.tif')

In [ ]:
# Calculate % in all classes

Ntotal=len(empty)+len(low_basket)+len(high_basket)+len(trash)

print('% of empty basket : ', len(empty)/Ntotal)
